## Setup

In [1]:
import sqlite3
import uuid

## Code practice

### Question 1

Create a database with three tables named `employees`, `salaries` and `departments`. The `employees` table should have these attributes *id*, *name* *department_id*. The `salaries` should have two attributes *id*, *employee_id* and *amount*. The `departments` should have two columns *id* and *dept_name*.

In [2]:
unique_filename = f'{uuid.uuid4().hex}'

In [3]:
conn = sqlite3.connect(f'/tmp/{unique_filename}_sample.db')

In [4]:
cursor = conn.cursor()

In [5]:
#create employees
cursor.execute('''CREATE TABLE IF NOT EXISTS employees
                  (id INTEGER PRIMARY KEY, name TEXT, department_id INTEGER)''')

# Create the salaries table
cursor.execute('''CREATE TABLE IF NOT EXISTS salaries
                  (id INTEGER PRIMARY KEY, employee_id INTEGER, amount INTEGER)''')

# Create the departments table
cursor.execute('''CREATE TABLE IF NOT EXISTS departments
                  (id INTEGER PRIMARY KEY , dept_name TEXT)''')

### Question 2

Write a MySQL query to add 10 rows of data into the `employees` table and the `salaries` table, and insert three departments into the departments table.

In [6]:
for i in range(1, 11):
    name = f"Employee {i}"
    department_id = i % 3 + 1
    cursor.execute("INSERT OR IGNORE INTO employees (name, department_id) VALUES (?, ?)", (name, department_id))

# Insert data into the salaries table
for i in range(1, 11):
    employee_id = i
    amount = 5000 + i * 1000
    cursor.execute("INSERT OR IGNORE INTO salaries (employee_id, amount) VALUES (?, ?)", (employee_id, amount))

# Insert data into the departments table
departments = [
    (1, 'HR'),
    (2, 'Engineering'),
    (3, 'Marketing')
]
cursor.executemany("INSERT OR IGNORE INTO departments (id, dept_name) VALUES (?, ?)", departments)

### Question 3

Retrieve, store and display all the data from `employees` table.

In [7]:
cursor.execute("SELECT * FROM employees")
employees_data = cursor.fetchall()
for row in employees_data:
    print(row)

(1, 'Employee 1', 2)
(2, 'Employee 2', 3)
(3, 'Employee 3', 1)
(4, 'Employee 4', 2)
(5, 'Employee 5', 3)
(6, 'Employee 6', 1)
(7, 'Employee 7', 2)
(8, 'Employee 8', 3)
(9, 'Employee 9', 1)
(10, 'Employee 10', 2)


### Question 4

Retrieve, store and display all the data from `salaries` table.

In [8]:
cursor.execute("SELECT * FROM salaries")
salaries_data = cursor.fetchall()
for row in salaries_data:
    print(row)

(1, 1, 6000)
(2, 2, 7000)
(3, 3, 8000)
(4, 4, 9000)
(5, 5, 10000)
(6, 6, 11000)
(7, 7, 12000)
(8, 8, 13000)
(9, 9, 14000)
(10, 10, 15000)


### Question 5

Retrieve, store and display all the data from `departments` table

In [9]:
cursor.execute("SELECT * FROM departments")
depart_data = cursor.fetchall()
for row in depart_data:
    print(row)

(1, 'HR')
(2, 'Engineering')
(3, 'Marketing')


### Question 6

Retrieve all employees along with their department names.

In [10]:
cursor.execute('''SELECT e.id, e.name, d.dept_name AS department_name
FROM employees e
JOIN departments d ON e.department_id = d.id''')
ret_1=cursor.fetchall()
for row in ret_1:
    print(row)

(1, 'Employee 1', 'Engineering')
(2, 'Employee 2', 'Marketing')
(3, 'Employee 3', 'HR')
(4, 'Employee 4', 'Engineering')
(5, 'Employee 5', 'Marketing')
(6, 'Employee 6', 'HR')
(7, 'Employee 7', 'Engineering')
(8, 'Employee 8', 'Marketing')
(9, 'Employee 9', 'HR')
(10, 'Employee 10', 'Engineering')


### Question 7

Retrieve the total salary amount for each employee.

In [11]:
cursor.execute('''SELECT e.id, e.name, SUM(s.amount) AS total_salary
FROM employees e
JOIN salaries s ON e.id = s.employee_id
GROUP BY e.id''')
ret_2=cursor.fetchall()
for row in ret_2:
    print(row)

(1, 'Employee 1', 6000)
(2, 'Employee 2', 7000)
(3, 'Employee 3', 8000)
(4, 'Employee 4', 9000)
(5, 'Employee 5', 10000)
(6, 'Employee 6', 11000)
(7, 'Employee 7', 12000)
(8, 'Employee 8', 13000)
(9, 'Employee 9', 14000)
(10, 'Employee 10', 15000)


### Question 8

Retrieve the department with the highest average salary.

In [12]:
cursor.execute('''SELECT d.dept_name AS department_name, AVG(s.amount) AS average_salary
FROM departments d
JOIN employees e ON d.id = e.department_id
JOIN salaries s ON e.id = s.employee_id
GROUP BY d.id
ORDER BY average_salary DESC
LIMIT 1''')
ret_3=cursor.fetchall()
for row in ret_3:
    print(row)

('HR', 11000.0)


### Question 9

Retrieve the employee(s) with the highest salary.

In [13]:
cursor.execute('''SELECT e.name, s.amount AS salary
FROM employees e
JOIN salaries s ON e.id = s.employee_id
WHERE s.amount = (SELECT MAX(amount) FROM salaries);
''')
ret_4=cursor.fetchall()
for row in ret_4:
    print(row)

('Employee 10', 15000)


### Question 10 (Views)

Write a MySQL query to create a view called "employee_details" that includes the employee's name, department name, and salary amount for each employee. Retrieve the details from the employees, departments, and salaries tables.

In [14]:
cursor.execute('''CREATE VIEW employee_details AS
SELECT e.name AS employee_name, d.dept_name AS department_name, s.amount AS salary_amount
FROM employees e
JOIN departments d ON e.department_id = d.id
JOIN salaries s ON e.id = s.employee_id''')
cursor.execute('''SELECT * FROM employee_details''')
ret_5=cursor.fetchall()
for row in ret_5:
    print(row)

('Employee 1', 'Engineering', 6000)
('Employee 2', 'Marketing', 7000)
('Employee 3', 'HR', 8000)
('Employee 4', 'Engineering', 9000)
('Employee 5', 'Marketing', 10000)
('Employee 6', 'HR', 11000)
('Employee 7', 'Engineering', 12000)
('Employee 8', 'Marketing', 13000)
('Employee 9', 'HR', 14000)
('Employee 10', 'Engineering', 15000)


### Question 11 (Window Functions)

Write a MySQL query to calculate the average salary for each department, along with the rank of each employee within their department based on their salary. Use a window function to achieve this, retrieving the department name, employee name, salary amount, average department salary, and employee rank.

In [15]:
cursor.execute('''SELECT department_name, employee_name, salary_amount, avg_department_salary,
       RANK() OVER (PARTITION BY department_name ORDER BY salary_amount DESC) AS employee_rank
FROM (
    SELECT e.name AS employee_name, d.dept_name AS department_name, s.amount AS salary_amount,
           AVG(s.amount) OVER (PARTITION BY d.dept_name) AS avg_department_salary
    FROM employees e
    JOIN departments d ON e.department_id = d.id
    JOIN salaries s ON e.id = s.employee_id
) AS subquery;
''')
ret_6=cursor.fetchall()
for row in ret_6:
  print(row)

('Engineering', 'Employee 10', 15000, 10500.0, 1)
('Engineering', 'Employee 7', 12000, 10500.0, 2)
('Engineering', 'Employee 4', 9000, 10500.0, 3)
('Engineering', 'Employee 1', 6000, 10500.0, 4)
('HR', 'Employee 9', 14000, 11000.0, 1)
('HR', 'Employee 6', 11000, 11000.0, 2)
('HR', 'Employee 3', 8000, 11000.0, 3)
('Marketing', 'Employee 8', 13000, 10000.0, 1)
('Marketing', 'Employee 5', 10000, 10000.0, 2)
('Marketing', 'Employee 2', 7000, 10000.0, 3)


### Question 12 (Subquery)

Write a MySQL query to retrieve the employee name, department name, and the highest salary in their department. Use a subquery to find the highest salary within each department, and then join the employees and departments tables to retrieve the corresponding employee and department details.

In [17]:
cursor.execute('''SELECT e.name AS employee_name, d.dept_name AS department_name, MAX(s.amount) AS highest_salary
FROM employees e
JOIN departments d ON e.department_id = d.id
JOIN salaries s ON e.id = s.employee_id
WHERE s.amount = (
    SELECT MAX(amount)
    FROM salaries
    WHERE employee_id = e.id
)
GROUP BY d.dept_name;
''')
ret_7=cursor.fetchall()
for row in ret_7:
    print(row)

('Employee 10', 'Engineering', 15000)
('Employee 9', 'HR', 14000)
('Employee 8', 'Marketing', 13000)
